# Using Supplychainpy with Pandas
** *by Kevin Fasusi* **

In workbook 0.0.4-Supplychainpy-Inventory-Analysis, we used standard Python types (dict, list, tuple, etc..). This analysis can also be conducted using a Pandas `DataFrame` giving us many more options for quick and easy analysis. 

To use Supplychainpy library with Pandas, we first need to import the right modules and read a CSV file to a Pandas DataFrame. The `%matplotlib inline` statement is used so we can see the matplotlib plots in the jupyter notebook.
First, we read the raw data into the Pandas `DataFrame`, as shown below.

In [1]:
%matplotlib inline

import matplotlib
import pandas as pd

from supplychainpy.model_inventory import analyse
from supplychainpy.model_demand import simple_exponential_smoothing_forecast
from supplychainpy.sample_data.config import ABS_FILE_PATH
from decimal import Decimal
raw_df =pd.read_csv(ABS_FILE_PATH['COMPLETE_CSV_SM'])

Passing a Pandas `DataFrame` as a keyword argument (df=) returns a `DataFrame` with the inventory profile analysed. Excluding the import statements, this can be achieved in 3 lines of code. There are several columns, so the print statement has been limited to a few ('sku','quantity_on_hand', 'excess_stock', 'shortages', 'ABC_XYZ_Classification').

In [8]:
orders_df = raw_df[['Sku','jan','feb','mar','apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']]
#orders_df.set_index('Sku')
analyse_kv =dict(
    df=raw_df, 
    start=1, 
    interval_length=12, 
    interval_type='months',
    z_value=Decimal(1.28), 
    reorder_cost=Decimal(400), 
    retail_price=Decimal(455), 
    file_type='csv', 
    currency='USD'
)
analysis_df = analyse( **analyse_kv)
print(analysis_df[['sku','quantity_on_hand', 'excess_stock', 'shortages', 'ABC_XYZ_Classification']])

          sku quantity_on_hand excess_stock shortages ABC_XYZ_Classification
0   KR202-209             1003            0      5969                     BY
1   KR202-210             3224            0         0                     CY
2   KR202-211              390            0      7099                     CY
3   KR202-212              390            0      7759                     CY
4   KR202-213             2095            0         0                     CY
5   KR202-214               55            0      5824                     CY
6   KR202-215             4308          732         0                     CY
7   KR202-216               34            0      6999                     CY
8   KR202-217              390            0      7245                     BY
9   KR202-218             3535            0         0                     CZ
10  KR202-219              334            0      5917                     CZ
11  KR202-220             3434            0         0                     BY

Before we can make a forecast, we need to select an SKU from the `analysis_df` variable. After selection, we slice the row to retrieve only orders data and convert to a `Series`. 

In [ ]:
row_ds = raw_df[raw_df['Sku']=='KR202-212'].squeeze()
print(row_ds[1:12])

Now that we have a `series` of orders data fro the SKU `KR202-212`, we can now perform a forecast using the `model_demand` module. We can perform a simple_exponential_smoothing_forecast by passing the forecasting function the orders data using the keyword parameter `ds=`.

In [ ]:
ses_df = simple_exponential_smoothing_forecast(ds=row_ds[1:12], length=12, smoothing_level_constant=0.5)
print(ses_df)

In [ ]:
print(ses_df.get('forecast', 'UNKNOWN'))

If we check the statistcs for the forecast we can see whether there is a linear trend and subsequently if the forecast is useful.

In [ ]:
print(ses_df.get('statistics', 'UNKNOWN'),'\n mape: {}'.format(ses_df.get('mape', 'UNKNOWN')))

The breakdown of the forecast is returned with the `forecast` and `statistics`.

In [ ]:
print(ses_df.get('forecast_breakdown', 'UNKNOWN'))

We can convert the `forecast_breakdown` back into a `DataFrame`.

In [ ]:
forecast_breakdown_df = pd.DataFrame(ses_df.get('forecast_breakdown', 'UNKNOWN'))
print(forecast_breakdown_df)

Let's look at the `demand` and the `one_step_forecast` in a chart.

In [ ]:
forecast_breakdown_df.plot(x='t', y=['one_step_forecast','demand'])

We can also create the data points for the regression line.

In [ ]:
regression = {'regression': [(ses_df.get('statistics')['slope']* i ) + ses_df.get('statistics')['intercept'] for i in range(1,12)]}
print(regression)

We can add the regression data points to the forecast breakdown DataFrame.

In [ ]:
forecast_breakdown_df['regression'] = regression.get('regression')
print(forecast_breakdown_df)

In [ ]:
forecast_breakdown_df.plot(x='t', y=['one_step_forecast','demand', 'regression'])

We have a choice now; we can use another alpha and repeat the analysis to reduce the Standard Error or use supplychainpy's `optimise=True` parameter to use an evolutionary algorithm and get closer to an optimal solution.

In [ ]:
opt_ses_df = simple_exponential_smoothing_forecast(ds=row_ds[1:12], length=12, smoothing_level_constant=0.4,optimise=True)
print(opt_ses_df)

In [ ]:
print(opt_ses_df.get('statistics', 'UNKNOWN'),'\n mape: {}'.format(opt_ses_df.get('mape', 'UNKNOWN')))

In [ ]:
print(opt_ses_df.get('forecast', 'UNKNOWN')) 

In [ ]:
optimised_regression = {'regression': [(opt_ses_df.get('statistics')['slope']* i ) + opt_ses_df.get('statistics')['intercept'] for i in range(1,12)]}
print(optimised_regression)

In [ ]:
opt_forecast_breakdown_df = pd.DataFrame(opt_ses_df.get('forecast_breakdown', 'UNKNOWN'))

In [ ]:
opt_forecast_breakdown_df['regression'] = optimised_regression.get('regression')
print(opt_forecast_breakdown_df)

In [ ]:
opt_forecast_breakdown_df.plot(x='t', y=['one_step_forecast','demand', 'regression'])